In [45]:
import pandas as pd
import numpy as np
import string
!pip install nltk
import nltk
nltk.download('punkt')
from nltk import word_tokenize, Counter
from nltk.corpus import stopwords
import itertools
import spacy
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gupta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Objective - Use the raw captions from scrapped data and convert it into a list of words and tf/idf


In [85]:
# reading the data
nike_data = pd.read_csv('nike_data.csv')
nike_data.head()

,Unnamed: 0,img_url,caption,n_likes_1000,n_comments,age
0,0,https://scontent-waw1-1.cdninstagram.com/v/t51...,“100% of myself is nothing compared to 1% of t...,218k,9590,1 day ago
1,1,https://scontent-waw1-1.cdninstagram.com/v/t51...,"Meet @azusa25nigo, the founder of @skate_girls...",77k,62680,2 days ago
2,2,https://scontent-waw1-1.cdninstagram.com/v/t51...,It takes courage to take the first step 🏃. Jus...,232k,46169,2 days ago
3,3,https://scontent-waw1-1.cdninstagram.com/v/t51...,“The climate crisis is affecting my sport and ...,158k,86967,6 days ago
4,4,https://scontent-waw1-1.cdninstagram.com/v/t51...,“People like to tell us what we can and can’t ...,251k,67604,1 week ago


In [86]:
nike_data.shape

(624, 6)

In [87]:
# function to lemmatize all words in captions
def lemmatization(text):
    text = nlp(text)
    text_lemma = [word.lemma_ for word in text]
    return " ".join(text_lemma)

In [88]:
# function to clean caption obtained from scrapper
def wrangle(text):
    text = text.replace('“', '"').replace('”','"').replace('’', "'")
    printable = set(string.printable)
    text = ''.join(filter(lambda x: x in printable, text))
   
    return text

In [89]:
# function to remove stop words and punctuations from the list of caption words
def remove_stopwords(lst):
    return [word for word in lst if 
            ( (word not in stopwords.words()) &
            (word not in list(string.punctuation)) &
            (word not in list(string.digits)))]

In [95]:
# applying the cleaning functions
def caption_cleaning(data):
    # subsetting captions only
    captions = data[['caption']]
    captions['caption'] = captions['caption'].astype(str).str.strip()
    captions['caption'] = captions['caption'].map(lambda s: wrangle(s))
    captions['caption_lemma'] = captions['caption'].map(lemmatization)

    # creating caption list
    captions['caption_list'] = captions['caption_lemma'].map(
        lambda row: word_tokenize(row.lower()))
    
    # removing stop words and punctuation
    captions['caption_list'] = captions['caption_list'].map(lambda row: remove_stopwords(row))
    
    return captions

In [96]:
# get tf idf dataframe
def tf_idf(col):
    # creating tf-idf vector
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(col.values)
    columns = vectorizer.get_feature_names()

    # creating tf idf df
    tf_idf_df = pd.DataFrame(X.toarray(), columns=columns)
    
    return tf_idf_df

In [107]:
# creating tf-idf vectors
def caption_tf_idf(captions):
    # creating a list of caption words
    caption_words_list = []
    for l in list(captions['caption_list'].values):
        caption_words_list = caption_words_list + l

    # removing duplicates 
    caption_words_list = list(set(caption_words_list))

    # removing words with length <= 2
    caption_words_list = [w for w in caption_words_list if len(w) > 2]

    # converting the text to list
    captions['caption_cleaned'] = captions['caption_list'].map(lambda lst: ' '.join(lst))
    
    # get tf idf vec
    caption_tf_idf = tf_idf(captions['caption_cleaned'])
    caption_words_list = list(set(caption_words_list).intersection(set(caption_tf_idf.columns)))
    caption_tf_idf = caption_tf_idf[caption_words_list]
    
    return caption_tf_idf

In [98]:
nike_captions = caption_cleaning(nike_data)

<ipython-input-95-2a8e923f7601>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  captions['caption'] = captions['caption'].astype(str).str.strip()
<ipython-input-95-2a8e923f7601>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  captions['caption'] = captions['caption'].map(lambda s: wrangle(s))
<ipython-input-95-2a8e923f7601>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [99]:
nike_captions

,caption,caption_lemma,caption_list
0,"""100% of myself is nothing compared to 1% of t...",""" 100 % of myself be nothing compare to 1 % of...","[``, 100, nothing, compare, team, ``, kipchoge..."
1,"Meet @azusa25nigo, the founder of @skate_girls...","Meet @azusa25nigo , the founder of @skate_girl...","[meet, azusa25nigo, founder, skate_girls_snap,..."
2,It takes courage to take the first step . Just...,it take courage to take the first step . just ...,"[courage, first, step, ask, azusa25nigo, azusa..."
3,"""The climate crisis is affecting my sport and ...",""" the climate crisis be affect my sport and at...","[``, climate, crisis, affect, sport, athlete, ..."
4,"""People like to tell us what we can and can't ...",""" People like to tell we what we can and ca n'...","[``, people, like, tell, n't, n't, hear, real,..."
...,...,...,...
619,Quick stop at Niketown London. Had to design a...,quick stop at Niketown London . have to design...,"[quick, stop, niketown, london, design, nikeid..."
620,Flyknit research at @1948London.,Flyknit research at @1948London .,"[flyknit, research, 1948london]"
621,A little testing turned into a full workout fo...,a little testing turn into a full workout for ...,"[little, testing, turn, full, workout, michell..."
622,A sport center in the center of sport. The Nik...,a sport center in the center of sport . the Ni...,"[sport, center, center, sport, nike+, fuelstat..."


In [108]:
nike_tf_idf = caption_tf_idf(nike_captions)

<ipython-input-107-aca970e00572>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  captions['caption_cleaned'] = captions['caption_list'].map(lambda lst: ' '.join(lst))


In [109]:
nike_tf_idf

,back,fast,passionate,train,asphalt,girls,starting,power,paulaaloe,marlen_esparza,...,breathe,statue,paralympic,thibautgrevet,work,sand,prelive,goals,rug,mat
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.114277,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.274966,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.133194,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.083238,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
620,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
621,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
622,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [111]:
nike_tf_idf.columns[:100]

Index(['back', 'fast', 'passionate', 'train', 'asphalt', 'girls', 'starting',
       'power', 'paulaaloe', 'marlen_esparza', 'purpose', 'improbable',
       'weekday', 'crowd', 'technically', 'hike', 'nothing', 'win', 'elina',
       'shine', 'werundc', 'dedication', 'igtv', 'makeitcount',
       'kevinhart4real', 'baltimore', 'paula', 'nadal', 'accordance', 'betrue',
       'easymoneysniper', 'reclaim', 'hour', 'lunar2', 'defeat', 'aim', 'keep',
       'niketennis', 'playinside', 'skater', 'tree', '100', 'overlook',
       'conclusion', 'ramla', 'anyway', 'nikesnowboardings', 'courtballistic',
       'java', 'lunarglide', 'believer', 'paulo', 'baylor', 'turf', 'ussoccer',
       'playoff', 'brigidkosgei', 'gurlstalk', 'family', 'forge', 'lunarlon',
       'stopasianhate', 'niketown', 'strategically', 'adar', 'serena', '1982',
       'meet', 'autumn', 'gearup', 'saturdays', 'indian', 'nouf', 'slogan',
       'shoe', 'ribbon', 'opponent', 'collide', 'dry', 'soul', 'throw',
       'appla